In [55]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import datetime as dt
import matplotlib.dates as mdates
import unidecode
from plotly.subplots import make_subplots
from get_data import read_data, get_data, download_csv
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd

## Primeiro, fazemos os plots em Matplotlib:

In [56]:
import git
import sys

root_dir = git.Repo('.', search_parent_directories=True).working_tree_dir
sys.path.insert(1, root_dir)

In [57]:
dados_covid_rj = pd.read_csv(f'{root_dir}/dados/RJ/cidade_rj/dados_covid/Dados_indiv_MRJ_covid19.csv', delimiter = ';')
dados_covid_rj = read_data(dados_covid_rj, 'prefeitura')


In [58]:
dados_covid_rj = dados_covid_rj.drop(columns = ['Faixa Etaria', 'Data_atualização'])

In [59]:
dados_covid_rj['AP Residência'].replace({'#N/D':'3.1'}, inplace = True)
dados_covid_rj['Bairro'].replace({'COMPLEXO DA MARE': 'MARE'}, inplace = True)

In [60]:
dados_pop_rj = pd.read_csv('pop_Bairros.csv', encoding = 'ISO-8859-1')
dados_pop_rj['NOME'] = [unidecode.unidecode(x.upper().strip()) for x in list(dados_pop_rj['NOME'])]
dados_covid_rj = dados_covid_rj[dados_covid_rj['Bairro'] != 'INDEFINIDO']
dados_covid_rj = dados_covid_rj[dados_covid_rj['Bairro'] != 'FORA DO MUNICÍPIO']

In [61]:
lst_bairros = dados_covid_rj['Bairro'].unique()
lst_bairros[121] = 'OSVALDO CRUZ'
lst_bairros[92] = 'FREGUESIA (JACAREPAGUA)'
lst_bairros[75] = 'FREGUESIA (ILHA)'
lst_bairros[108] = 'CAVALCANTI'
lst_bairros[118] = 'RICARDO DE ALBUQUERQUE'

In [62]:
data_start = '15-03-2020'
data_end = '03-05-2020'
start = dt.datetime.strptime(data_start, "%d-%m-%Y")
then = dt.datetime.strptime(data_end, "%d-%m-%Y")
days = mdates.drange(start,then,dt.timedelta(days=1))
dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]

In [63]:
lst_dic = {}
for b in lst_bairros:
    lst_dic.update({b: get_data(b, dados_covid_rj, 'prefeitura', T_fim = data_end, T_start = data_start, to_print = False)})

In [64]:
lst_acul = []

In [65]:
dic_acum = {'Data': dias}

In [66]:
dic_acum.update({b: lst_dic[b][0] for b in lst_bairros})

In [67]:
df_acum = pd.DataFrame.from_dict(dic_acum)

In [68]:
dic_novos = {'Data': dias}

In [69]:
dic_novos.update({b: lst_dic[b][1] for b in lst_bairros})

In [70]:
df_novos = pd.DataFrame.from_dict(dic_novos)

In [71]:
n_semanas = 7

listas_semanas_acumul =[[] for i in range(n_semanas)]
listas_semanas_acumul_obitos =[[] for i in range(n_semanas)]


lst_novos = [[] for i in range(n_semanas)]
lst_novos_obitos = [[] for i in range(n_semanas)]
for b in lst_bairros:
    b_novos = get_data(df = dados_covid_rj, local = b, fonte = 'prefeitura', T_fim = data_end, T_start = data_start, to_print = False)[1]
    b_novos_obitos = get_data(df = dados_covid_rj[dados_covid_rj['Situação atual'] == 'óbito'], local = b, fonte = 'prefeitura', T_fim = data_end, T_start = data_start, to_print = False)[1]
    
    for j,l in enumerate(listas_semanas_acumul):
        l.append(get_data(df = dados_covid_rj, local = b, fonte = 'prefeitura', T_fim = data_end, T_start = data_start, to_print = False)[0][-7*j-1])
        listas_semanas_acumul_obitos[j].append(get_data(df = dados_covid_rj[dados_covid_rj['Situação atual'] == 'óbito'], local = b, fonte = 'prefeitura', T_fim = data_end, T_start = data_start, to_print = False)[0][-7*j-1])
        lst_novos[j].append(sum(b_novos[7*j:7*(j+1) - 1]))
        lst_novos_obitos[j].append(sum(b_novos_obitos[7*j:7*(j+1) - 1]))


In [72]:
#dados_pop_rj

In [73]:
dic_covid = {'Acumulados semana '+str(n_semanas-j): listas_semanas_acumul[j] for j in range(n_semanas)}
dic_covid.update({'Acumulados óbitos semana '+str(n_semanas-j): listas_semanas_acumul_obitos[j] for j in range(n_semanas)})
dic_covid.update({'Bairros': lst_bairros})
dic_covid.update({'Novos semana '+str(j+1): lst_novos[j] for j in range(n_semanas)})
dic_covid.update({'Novos óbitos semana '+str(j+1): lst_novos_obitos[j] for j in range(n_semanas)})
pop_bairros = [dados_pop_rj['poptotal'].values[list(dados_pop_rj['NOME']).index(b)] for b in lst_bairros]
ap_bairros = [dados_pop_rj['AREA_PLANE'].values[list(dados_pop_rj['NOME']).index(b)] for b in lst_bairros]
df = pd.DataFrame.from_dict(dic_covid)
df['pop'] = pop_bairros
df['ap'] = ap_bairros

In [74]:
df.columns

Index(['Acumulados semana 7', 'Acumulados semana 6', 'Acumulados semana 5',
       'Acumulados semana 4', 'Acumulados semana 3', 'Acumulados semana 2',
       'Acumulados semana 1', 'Acumulados óbitos semana 7',
       'Acumulados óbitos semana 6', 'Acumulados óbitos semana 5',
       'Acumulados óbitos semana 4', 'Acumulados óbitos semana 3',
       'Acumulados óbitos semana 2', 'Acumulados óbitos semana 1', 'Bairros',
       'Novos semana 1', 'Novos semana 2', 'Novos semana 3', 'Novos semana 4',
       'Novos semana 5', 'Novos semana 6', 'Novos semana 7',
       'Novos óbitos semana 1', 'Novos óbitos semana 2',
       'Novos óbitos semana 3', 'Novos óbitos semana 4',
       'Novos óbitos semana 5', 'Novos óbitos semana 6',
       'Novos óbitos semana 7', 'pop', 'ap'],
      dtype='object')

## Fazendo o html para bairros e áreas de planejamento do RJ:

In [75]:
import plotly.graph_objects as go

In [76]:
with open(f'{root_dir}/dados/RJ/cidade_rj/dados_geo/geojs_Bairros.json', encoding = 'ISO-8859-1') as geofile:
    j_file = json.load(geofile)
with open(f'{root_dir}/dados/RJ/cidade_rj/dados_geo/geojs_AP.json', encoding = 'ISO-8859-1') as geofile_AP:
    j_file_AP = json.load(geofile_AP)
    

In [77]:
## áreas de planejamento do rj
AP1 = df[df['ap'] == 1]['Bairros'].values
AP2 = df[df['ap'] == 2]['Bairros'].values
AP3 = df[df['ap'] == 3]['Bairros'].values
AP4 = df[df['ap'] == 4]['Bairros'].values
AP5 = df[df['ap'] == 5]['Bairros'].values
aps = [AP1, AP2, AP3, AP4, AP5]

In [78]:
df

,Acumulados semana 7,Acumulados semana 6,Acumulados semana 5,Acumulados semana 4,Acumulados semana 3,Acumulados semana 2,Acumulados semana 1,Acumulados óbitos semana 7,Acumulados óbitos semana 6,Acumulados óbitos semana 5,...,Novos semana 7,Novos óbitos semana 1,Novos óbitos semana 2,Novos óbitos semana 3,Novos óbitos semana 4,Novos óbitos semana 5,Novos óbitos semana 6,Novos óbitos semana 7,pop,ap
0,362,335,291,198,144,103,49,42,36,29,...,24,0,3,3,7,8,6,6,135924,4
1,74,73,66,54,41,30,13,3,3,3,...,1,1,0,0,0,1,0,0,21198,2
2,113,82,60,27,14,10,9,31,23,15,...,22,0,1,1,3,8,7,7,217333,5
3,20,17,12,4,1,0,0,4,4,3,...,2,0,0,1,0,2,1,0,37856,4
4,30,27,23,17,9,6,4,2,2,1,...,3,0,0,0,1,0,1,0,24057,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,2,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2016,3
155,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2749,1
156,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,9202,3
157,2,1,1,1,1,0,0,1,0,0,...,1,0,0,0,0,0,0,1,13184,5


In [79]:
vals_acum = []
vals_acum_obitos = []
vals_novos = []
vals_novos_obitos = []
for i in range(1, 6):
    vals_acum += [[df[df['ap'] == i][f'Acumulados semana %s' %(j)].sum() for j in range(1, 1 + n_semanas)]]
    vals_acum_obitos += [[df[df['ap'] == i][f'Acumulados óbitos semana %s' %(j)].sum() for j in range(1, 1 + n_semanas)]]
    vals_novos += [[df[df['ap'] == i][f'Novos semana %s' %(j)].sum() for j in range(1, 1 + n_semanas)]]
    vals_novos_obitos += [[df[df['ap'] == i][f'Novos óbitos semana %s' %(j)].sum() for j in range(1, 1 + n_semanas)]]


In [80]:
vals_acum

[[24, 39, 78, 145, 225, 294, 335],
 [243, 493, 713, 1074, 1504, 1851, 2062],
 [127, 212, 430, 820, 1393, 1807, 2104],
 [101, 197, 322, 494, 747, 920, 1038],
 [72, 104, 193, 411, 716, 977, 1142]]

In [81]:
pop1  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 1]['poptotal'])
pop2  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 2]['poptotal'])
pop3  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 3]['poptotal'])
pop4  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 4]['poptotal'])
pop5  = sum(dados_pop_rj[dados_pop_rj['AREA_PLANE'] == 5]['poptotal'])

In [82]:
pop1, pop2, pop3, pop4, pop5
pops = np.array([pop1, pop2, pop3, pop4, pop5])

In [83]:
#Casos0_AP, Casos1_AP, Casos2_AP, Casos3_AP = [], [], [], []
#CasosSemana0_AP, CasosSemana1_AP, CasosSemana2_AP, CasosSemana3_AP = [], [], [], []

In [84]:
v_acum = []
v_acum_obitos = []
v_novos = []
v_novos_obitos = []
for i in range(len(vals_acum[0])):
    v = []
    v_obitos = []
    novos = []
    novos_obitos = []
    for j in range(len(vals_acum)):
        v.append(vals_acum[j][i])
        v_obitos.append(vals_acum_obitos[j][i])
        novos.append(vals_novos[j][i])
        novos_obitos.append(vals_novos_obitos[j][i])
    v_acum.append(v)
    v_acum_obitos.append(v_obitos)
    v_novos.append(novos)
    v_novos_obitos.append(novos_obitos)    

In [85]:
dic_ap = {'AP': [1, 2, 3, 4, 5], 'pop':pops}
dic_ap.update({f'Acumulados semana %s' %(i): v_acum[i-1] for i in range(1, n_semanas+1)})
dic_ap.update({f'Acumulados óbitos semana %s' %(i): v_acum_obitos[i-1] for i in range(1, n_semanas+1)})
dic_ap.update({f'Novos semana %s' %(i): v_novos[i-1] for i in range(1, n_semanas+1)})
dic_ap.update({f'Novos óbitos semana %s' %(i): v_novos_obitos[i-1] for i in range(1, n_semanas+1)})
df_AP = pd.DataFrame(dic_ap)

In [86]:
novos_nomes = {}
for v in j_file['features']:
    for b in df['Bairros']:
        if b == 'FREGUESIA (ILHA)':
            novos_nomes.update({b:'Freguesia'})
        else:
            new_v = unidecode.unidecode(str(v['properties']['NOME']).upper())
            if new_v == b:
                novos_nomes.update({b:v['properties']['NOME']})


In [87]:
novos_nomes_col = [novos_nomes[b] for b in df['Bairros']]

In [88]:
df['nomes_certos'] = novos_nomes_col

In [89]:
objectid_dic = {v['properties']['NOME']:v['properties']['OBJECTID'] for v in j_file['features']}

In [90]:
objectid_dic.keys()

dict_keys(['Saúde', 'Gamboa', 'Santo Cristo', 'Caju', 'Centro', 'Catumbi', 'Rio Comprido', 'Cidade Nova', 'Estácio', 'São Cristóvão', 'Mangueira', 'Benfica', 'Paquetá', 'Santa Teresa', 'Flamengo', 'Glória', 'Laranjeiras', 'Catete', 'Cosme Velho', 'Botafogo', 'Humaitá', 'Urca', 'Leme', 'Copacabana', 'Ipanema', 'Leblon', 'Lagoa', 'Jardim Botânico', 'Gávea', 'Vidigal', 'São Conrado', 'Praça da Bandeira', 'Tijuca', 'Alto da Boa Vista', 'Maracanã', 'Vila Isabel', 'Andaraí', 'Grajaú', 'Manguinhos', 'Bonsucesso', 'Ramos', 'Olaria', 'Penha', 'Penha Circular', 'Brás de Pina', 'Cordovil', 'Parada de Lucas', 'Vigário Geral', 'Jardim América', 'Higienópolis', 'Jacaré', 'Maria da Graça', 'Del Castilho', 'Inhaúma', 'Engenho da Rainha', 'Tomás Coelho', 'São Francisco Xavier', 'Rocha', 'Riachuelo', 'Sampaio', 'Engenho Novo', 'Lins de Vasconcelos', 'Méier', 'Todos os Santos', 'Cachambi', 'Engenho de Dentro', 'Água Santa', 'Encantado', 'Piedade', 'Abolição', 'Pilares', 'Vila Kosmos', 'Vicente de Carvalh

In [91]:
objectid = []
for b in novos_nomes_col:
    if b == 'Freguesia':
        objectid.append(objectid_dic['Freguesia (Ilha)'])
    else:
        objectid.append(objectid_dic[b])

In [92]:
df['objectid'] = objectid

In [93]:
df

,Acumulados semana 7,Acumulados semana 6,Acumulados semana 5,Acumulados semana 4,Acumulados semana 3,Acumulados semana 2,Acumulados semana 1,Acumulados óbitos semana 7,Acumulados óbitos semana 6,Acumulados óbitos semana 5,...,Novos óbitos semana 2,Novos óbitos semana 3,Novos óbitos semana 4,Novos óbitos semana 5,Novos óbitos semana 6,Novos óbitos semana 7,pop,ap,nomes_certos,objectid
0,362,335,291,198,144,103,49,42,36,29,...,3,3,7,8,6,6,135924,4,Barra da Tijuca,471
1,74,73,66,54,41,30,13,3,3,3,...,0,0,0,1,0,0,21198,2,Lagoa,468
2,113,82,60,27,14,10,9,31,23,15,...,1,1,3,8,7,7,217333,5,Santa Cruz,387
3,20,17,12,4,1,0,0,4,4,3,...,0,1,0,2,1,0,37856,4,Tanque,429
4,30,27,23,17,9,6,4,2,2,1,...,0,0,1,0,1,0,24057,2,Catete,447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,2,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2016,3,Zumbi,353
155,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,2749,1,Saúde,425
156,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,9202,3,Parque Colúmbia,341
157,2,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,1,13184,5,Vila Militar,366


In [94]:
len(j_file['features']), len(df['nomes_certos'])

(163, 159)

In [95]:
df.to_csv('dados-certos-11abr.csv')

In [96]:
df

,Acumulados semana 7,Acumulados semana 6,Acumulados semana 5,Acumulados semana 4,Acumulados semana 3,Acumulados semana 2,Acumulados semana 1,Acumulados óbitos semana 7,Acumulados óbitos semana 6,Acumulados óbitos semana 5,...,Novos óbitos semana 2,Novos óbitos semana 3,Novos óbitos semana 4,Novos óbitos semana 5,Novos óbitos semana 6,Novos óbitos semana 7,pop,ap,nomes_certos,objectid
0,362,335,291,198,144,103,49,42,36,29,...,3,3,7,8,6,6,135924,4,Barra da Tijuca,471
1,74,73,66,54,41,30,13,3,3,3,...,0,0,0,1,0,0,21198,2,Lagoa,468
2,113,82,60,27,14,10,9,31,23,15,...,1,1,3,8,7,7,217333,5,Santa Cruz,387
3,20,17,12,4,1,0,0,4,4,3,...,0,1,0,2,1,0,37856,4,Tanque,429
4,30,27,23,17,9,6,4,2,2,1,...,0,0,1,0,1,0,24057,2,Catete,447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,2,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2016,3,Zumbi,353
155,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,2749,1,Saúde,425
156,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,9202,3,Parque Colúmbia,341
157,2,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,1,13184,5,Vila Militar,366


In [97]:
data_novos = [go.Choropleth(z = df[f'Novos semana %s' %(i+1)]*1e4/df['pop'], 
                          geojson = j_file,
                          locations= df['nomes_certos'],
                          featureidkey = 'properties.NOME',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = (i==0),
                          name = '',
                          zmin = 0,
                          zmax = 7,
                          customdata = df[f'Novos semana %s' %(i+1)],
 hovertemplate = "%{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}",
                          text = df['nomes_certos'])
                          for i in range(n_semanas)]
data_novos_obitos = [go.Choropleth(z = df[f'Novos óbitos semana %s' %(i+1)]*1e4/df['pop'], 
                          geojson = j_file,
                          locations= df['nomes_certos'],
                          featureidkey = 'properties.NOME',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = (i==0),
                          name = '',
                          zmin = 0,
                          zmax = 2,
                          customdata = df[f'Novos óbitos semana %s' %(i+1)],
 hovertemplate = "%{text} <br>Óbitos rel.: %{z:.3f} </br> Óbitos: %{customdata:,f}",
                          text = df['nomes_certos'])
                          for i in range(n_semanas)]

data_novos_AP = [go.Choropleth(z = 1e4*df_AP[f'Novos semana %s' %(i+1)]/df_AP['pop'], 
                          geojson = j_file_AP,
                          locations= df_AP['AP'],
                          featureidkey = 'properties.CODAPNUM',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          name = '',
                          zmin = 0,
                          zmax = 3,
                          visible = False,
            customdata = df_AP[f'Novos semana %s'%(i+1)],
 hovertemplate = "AP: %{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}",
                          text = df_AP['AP'])
                          for i in range(n_semanas)]

data_acumul = [go.Choropleth(z = 1e4*df[f'Acumulados semana %s'%(abs(i+1))]/df['pop'], 
                          geojson = j_file,
                          locations= df['nomes_certos'],
                          featureidkey = 'properties.NOME',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = False,
                          name = '',
                          zmin = 0,
                          zmax = 31,
                customdata = df[f'Acumulados semana %s'%(i+1)],
 hovertemplate = "%{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}", 
                          text = df['nomes_certos']
                          )
                          for i in range(n_semanas)]

data_acumul_obitos = [go.Choropleth(z = 1e4*df[f'Acumulados óbitos semana %s'%(abs(i+1))]/df['pop'], 
                          geojson = j_file,
                          locations= df['nomes_certos'],
                          featureidkey = 'properties.NOME',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = False,
                          name = '',
                          zmin = 0,
                          zmax = 7,
                customdata = df[f'Acumulados óbitos semana %s'%(i+1)],
 hovertemplate = "%{text} <br>Óbitos rel.: %{z:.3f} </br> Óbitos: %{customdata:,f}", 
                          text = df['nomes_certos']
                          )
                          for i in range(n_semanas)]

data_acumul_AP = [go.Choropleth(z = 1e4*df_AP[f'Acumulados semana %s'%(i+1)]/df_AP['pop'], 
                          geojson = j_file_AP,
                          locations= df_AP['AP'],
                          featureidkey = 'properties.CODAPNUM',
                          colorscale = "ylorrd",
                          colorbar_title = 'rel. 10 mil hab.',
                          visible = False,
                          zmin = 0,
                          zmax = 13,
                          name = '',
                customdata = df_AP[f'Acumulados semana %s'%(i+1)],
 hovertemplate = "AP: %{text} <br>Casos rel.: %{z:.3f} </br> Casos: %{customdata:,f}",
                          text = df_AP['AP'])
                          for i in range(n_semanas)]

    
data = data_novos + data_acumul + data_novos_AP + data_acumul_AP

In [98]:
dias = ['15/03', '22/03', '29/03', '05/04', '12/04', '19/04', '26/04', '03/05']
dias_fim = ['21/03', '28/03', '04/04', '11/04', '18/04', '25/04', '02/05']

In [99]:
titles = [f'Casos <b>novos</b> de {dias[i]} até {dias_fim[i]} <br> nos bairros do Rio de Janeiro </br>' for i in range(n_semanas)] + [f'Casos <b>acumulados</b> até {dias_fim[i]} <br> nos bairros do Rio de Janeiro </br>' for i in range(n_semanas)] +[f'Casos <b>novos</b> de {dias[i]} até {dias_fim[i]} <br> nas áreas de planejamento do Rio de Janeiro </br>' for i in range(n_semanas)] + [f'Casos <br>acumulados</br> até {dias_fim[i]} <br> nas áreas de planejamento do Rio de Janeiro </br>' for i in range(n_semanas)]
titles_obitos = [f'Óbitos <b>novos</b> de {dias[i]} até {dias_fim[i]} <br> nos bairros do Rio de Janeiro </br>' for i in range(n_semanas)] + [f'<b>Total</b> de óbitos até {dias_fim[i]} <br> nos bairros do Rio de Janeiro </br>' for i in range(n_semanas)] +[f'Óbitos <b>novos</b> de {dias[i]} até {dias_fim[i]} <br> nas áreas de planejamento do Rio de Janeiro </br>' for i in range(n_semanas)] + [f'<b>Total</b> de óbitos até {dias_fim[i]} <br> nas áreas de planejamento do Rio de Janeiro </br>' for i in range(n_semanas)]
labels = [f'{dias[i]} até {dias_fim[i]}' for i in range(n_semanas)] + [f'até {dias_fim[i]}' for i in range(n_semanas)] + [f'{dias[i]} até {dias_fim[i]} (AP)' for i in range(n_semanas)] + [f'até {dias_fim[i]} (AP)' for i in range(n_semanas)]

In [100]:
updatemenus_novos = list([dict(active = 0,
                         buttons = list([dict(label = labels[i],
                                              method = 'update', 
                                              args = [{'visible':  [ j == i for j in range(len(data_novos))] },
                                                      {'title': titles[i]}
                                                      ]) for i in range(len(data_novos))])) ])

In [47]:
#get_data(local='BARRA DA TIJUCA', df=dados_covid_rj, fonte='prefeitura', T_start=data_start, T_fim=data_end)

In [48]:
%%time
fig = go.Figure(data= data_novos)
fig.update_layout(updatemenus = updatemenus_novos, 
    title = dict(text = 'Casos <b>novos</b> de 15/03 até 21/03<br> nos bairros do Rio de Janeiro </br>',
                 x = 0.5))
fig.update_geos(fitbounds="locations", visible=False)
#fig.show()
#with open("mapa_bairro_confirmados_novos.html", "w") as f:
    f.write(fig.to_html())

IndentationError: unexpected indent (<unknown>, line 8)

In [ ]:
updatemenus_acumul = list([dict(active = 0,
                         buttons = list([dict(label = labels[i+len(data_novos)],
                                              method = 'update', 
                                              args = [{'visible':  [ j == i for j in range(len(data_acumul))] },
                                                      {'title': titles[i+len(data_novos)]}
                                                      ]) for i in range(len(data_acumul))])) ])

In [ ]:
%%time
data_acumul[0].visible = True
fig = go.Figure(data= data_acumul)
fig.update_layout(updatemenus = updatemenus_acumul, 
    title = dict(text = 'Casos <b>acumulados</b> até 21/03<br> nos bairros do Rio de Janeiro </br>',
                 x = 0.5))
fig.update_geos(fitbounds="locations", visible=False)
#fig.show()
#with open("mapa_bairro_confirmados_acumulados.html", "w") as f:
    f.write(fig.to_html())

In [ ]:
updatemenus_novos_obitos = list([dict(active = 0,
                         buttons = list([dict(label = labels[i],
                                              method = 'update', 
                                              args = [{'visible':  [ j == i for j in range(len(data_novos_obitos))] },
                                                      {'title': titles_obitos[i]}
                                                      ]) for i in range(len(data_novos_obitos))])) ])

In [ ]:
%%time
fig = go.Figure(data= data_novos_obitos)
fig.update_layout(updatemenus = updatemenus_novos_obitos, 
    title = dict(text = 'Óbitos <b>novos</b> de 15/03 até 21/03<br> nos bairros do Rio de Janeiro </br>',
                 x = 0.5))
fig.update_geos(fitbounds="locations", visible=False)
#fig.show()
#with open("mapa_bairro_obitos_novos.html", "w") as f:
#    f.write(fig.to_html())

In [ ]:
updatemenus_acumul_obitos = list([dict(active = 0,
                         buttons = list([dict(label = labels[i+len(data_novos)],
                                              method = 'update', 
                                              args = [{'visible':  [ j == i for j in range(len(data_acumul))] },
                                                      {'title': titles_obitos[i+len(data_novos)]}
                                                      ]) for i in range(len(data_acumul_obitos))])) ])

In [ ]:
%%time
data_acumul_obitos[0].visible = True
fig = go.Figure(data= data_acumul_obitos)
fig.update_layout(updatemenus = updatemenus_acumul_obitos, 
    title = dict(text = '<b>Total</b> de óbitos até 21/03<br> nos bairros do Rio de Janeiro </br>',
                 x = 0.5))
fig.update_geos(fitbounds="locations", visible=False)
#fig.show()
#with open("mapa_bairro_obitos_acumulados.html", "w") as f:
#    f.write(fig.to_html())